In [1]:
import pandas as pd
import os
from glob import glob
from pathlib import Path
from pymatgen.io.cif import CifParser #cif
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer#sp
from pymatgen.core.ion import Ion #ion
from pymatgen.core.structure import Structure
from pymatgen.analysis.local_env import CrystalNN #co-number
import warnings 
warnings.filterwarnings("ignore")
Crystal_nn = CrystalNN()
cif_files = glob('/*.cif')
datas = pd.read_csv('formula.csv')
print(datas)
print(cif_files)

           mp-id     formula
0     mp-1372341   Mg2CuSbO6
1     mp-1047139     Ca2VWO6
2     mp-1184736      HoCuO3
3      mp-776476      CdFeO3
4     mp-1188081      PuNiO3
...          ...         ...
4183    mp-16978  Ba3MgSb2O9
4184    mp-12642   Ba2TmNbO6
4185   mp-558696      CuTeO3
4186  mp-1016875      CdRhO3
4187  mp-1519256   Ba2TbSnO6

[4188 rows x 2 columns]
['/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/Mg2CuSbO6(mp-1372341).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/Ca2VWO6(mp-1047139).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/HoCuO3(mp-1184736).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/CdFeO3(mp-776476).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/PuNiO3(mp-1188081).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/Sr2CdWO6(mp-19014).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/LiPO3(mp-1202801).cif', '/Users/yangmingyue/Desktop/ABO365/模型应用新新/符合要求Structure/C

In [2]:
import notebook
notebook.__version__

'7.1.1'

In [9]:
import pandas as pd
import os
from glob import glob
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.ion import Ion
from pymatgen.core.structure import Structure
from pymatgen.analysis.local_env import CrystalNN
import warnings

warnings.filterwarnings("ignore")


Crystal_nn = CrystalNN()


cif_files = glob('*.cif')
datas = pd.read_csv('formula.csv')


output_folder = '/coordination number/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


cal = 1
for data in datas.values.tolist():
    mp, formula = data
    name = f"{formula}({mp}).cif"
    for cif_file in cif_files:
        if os.path.basename(cif_file) == name:
            coordination_num = pd.DataFrame()
            coordination_num_final = pd.DataFrame()
            parser = CifParser(cif_file) 
            structure = parser.get_structures()[0]  
            formula = structure.composition.reduced_formula
            sym_finder = SpacegroupAnalyzer(structure)
            compound = formula
            comp = Ion.from_formula(compound).composition  
            comp_dict = comp.as_dict()  
            space_group_number = sym_finder.get_space_group_number()
            print(f'{cal}. 目前化合物是{formula}')
            cal += 1

            for ion, count in comp_dict.items():
                indices = pd.DataFrame()
                print(f'正在处理{ion}')
                dope = count - int(count)
                if dope == 0:
                    for i, site in enumerate(structure):
                        if len(site.species_string) > 2: 
                            if len(ion) == 2:
                                ion_judge = site.species_string[0] + site.species_string[1]
                                if ion_judge == ion:
                                    indices = indices.append({'id': i}, ignore_index=True)
                            else:
                                ion_judge = site.species_string[0]
                                if ion_judge == ion:
                                    indices = indices.append({'id': i}, ignore_index=True)
                        else:
                            if site.species_string == ion:
                                indices = indices.append({'id': i}, ignore_index=True)
                else:
                    for i, site in enumerate(structure):
                        if len(site.species_string) > 4: 
                            dope_element = site.species_string.split(", ")
                            if len(ion) == 2:
                                for x in dope_element:
                                    element = x[0] + x[1]
                                    if element == ion:
                                        indices = indices.append({'id': i}, ignore_index=True)
                            else:
                                for x in dope_element:
                                    element = x[0]
                                    if element == ion:
                                        indices = indices.append({'id': i}, ignore_index=True)

                indices['id'] = indices['id'].astype(int)
                indices = indices['id'].tolist()

               
                CNs = []
                for i in indices:
                    try:
                        cn = Crystal_nn.get_cn(structure, i)
                        CNs.append(cn)
                    except ValueError as e:
                        print(f"Error processing site {i} in {formula}: {e}")
                        continue

                for i, cn in zip(indices, CNs):
                    coordination_num = coordination_num.append({'formula': formula, 'id': i, 'ion': ion, 'count': count, 'cn': cn}, ignore_index=True)

            coordination_num['id_ion_total'] = coordination_num.groupby('id')['count'].transform('sum')
            for row in coordination_num.values.tolist():
                formula, id_1, ion, count, cn, id_ion_total = row
                id_ion_ratio = count / id_ion_total
                coordination_num_final = coordination_num_final.append({'formula': formula, 'id': id_1, 'ion': ion, 
                                                                        'count': count, 'cn': cn, 'id_ion_ratio': id_ion_ratio}, ignore_index=True)

            coordination_num_final = coordination_num_final.sort_values(by='id', ascending=True)
            output_file = os.path.join(output_folder, f'{formula}({mp}).csv')
            coordination_num_final.to_csv(output_file, index=False)
            print(f"Processed {formula} and saved to {output_file}")

print("Processing complete.")

1. 目前化合物是Mg2CuSbO6
正在处理Mg
正在处理Cu
正在处理Sb
正在处理O
Processed Mg2CuSbO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Mg2CuSbO6(mp-1372341).csv
2. 目前化合物是Ca2VWO6
正在处理Ca
正在处理V
正在处理W
正在处理O
Processed Ca2VWO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ca2VWO6(mp-1047139).csv
3. 目前化合物是HoCuO3
正在处理Ho
正在处理Cu
正在处理O
Processed HoCuO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/HoCuO3(mp-1184736).csv
4. 目前化合物是CdFeO3
正在处理Cd
正在处理Fe
正在处理O
Processed CdFeO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/CdFeO3(mp-776476).csv
5. 目前化合物是PuNiO3
正在处理Pu
正在处理Ni
正在处理O
Processed PuNiO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/PuNiO3(mp-1188081).csv
6. 目前化合物是Sr2CdWO6
正在处理Sr
正在处理Cd
正在处理W
正在处理O
Processed Sr2CdWO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Sr2CdWO6(mp-19014).csv
7. 目前化合物是LiPO3
正在处理Li
正在处理P
正在处理O
Processed LiPO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/LiPO3(mp-1202801).csv
8. 目前化合物是Ca2LaSbO6
正在处理Ca
正在处理La


spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).


505. 目前化合物是Ba2YZrO6
正在处理Ba
正在处理Y
正在处理Zr
正在处理O
Processed Ba2YZrO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ba2YZrO6(mp-1205685).csv
506. 目前化合物是Nd2CoRuO6
正在处理Nd
正在处理Co
正在处理Ru
正在处理O
Processed Nd2CoRuO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Nd2CoRuO6(mp-1210035).csv
507. 目前化合物是LaAlO3
正在处理La
正在处理Al
正在处理O
Processed LaAlO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/LaAlO3(mp-771703).csv
508. 目前化合物是Ba2EuReO6
正在处理Ba
正在处理Eu
正在处理Re
正在处理O
Processed Ba2EuReO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ba2EuReO6(mp-504674).csv
509. 目前化合物是Dy2MnNiO6
正在处理Dy
正在处理Mn
正在处理Ni
正在处理O
Processed Dy2MnNiO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Dy2MnNiO6(mp-1104659).csv
510. 目前化合物是ScRhO3
正在处理Sc
正在处理Rh
正在处理O
Processed ScRhO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/ScRhO3(mp-1196890).csv
511. 目前化合物是RbAuO3
正在处理Rb
正在处理Au
正在处理O
Processed RbAuO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/RbAuO3(mp-975

spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).


Processed LaMnO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/LaMnO3(mp-1278738).csv
537. 目前化合物是Ba2CoWO6
正在处理Ba
正在处理Co
正在处理W
正在处理O
Processed Ba2CoWO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ba2CoWO6(mp-22112).csv
538. 目前化合物是Sr2GdWO6
正在处理Sr
正在处理Gd
正在处理W
正在处理O
Processed Sr2GdWO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Sr2GdWO6(mp-1522401).csv
539. 目前化合物是Ca2CoSbO6
正在处理Ca
正在处理Co
正在处理Sb
正在处理O
Processed Ca2CoSbO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ca2CoSbO6(mp-1379512).csv
540. 目前化合物是CaCO3
正在处理Ca
正在处理C
正在处理O
Processed CaCO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/CaCO3(mp-561412).csv
541. 目前化合物是EuUO3
正在处理Eu
正在处理U
正在处理O
Processed EuUO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/EuUO3(mp-1213318).csv
542. 目前化合物是NClO3
正在处理N
正在处理Cl
正在处理O
Processed NClO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/NClO3(mp-754712).csv
543. 目前化合物是BaTiO3
正在处理Ba
正在处理Ti
正在处理O
Processed BaTiO3 and sav

...

Processed Ba3Nb2CoO9 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ba3Nb2CoO9(mp-31514).csv
3052. 目前化合物是CaCoO3
正在处理Ca
正在处理Co
正在处理O
Processed CaCoO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/CaCoO3(mp-1399298).csv
3053. 目前化合物是RbNiO3
正在处理Rb
正在处理Ni
正在处理O
Processed RbNiO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/RbNiO3(mp-975373).csv
3054. 目前化合物是CsRuO3
正在处理Cs
正在处理Ru
正在处理O
Processed CsRuO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/CsRuO3(mp-1183971).csv
3055. 目前化合物是NdTaO3
正在处理Nd
正在处理Ta
正在处理O
Processed NdTaO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/NdTaO3(mp-1186324).csv
3056. 目前化合物是DyCrO3
正在处理Dy
正在处理Cr
正在处理O
Processed DyCrO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/DyCrO3(mp-19381).csv
3057. 目前化合物是MgBiO3
正在处理Mg
正在处理Bi
正在处理O
Processed MgBiO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/MgBiO3(mp-1376863).csv
3058. 目前化合物是Dy2TiCuO6
正在处理Dy
正在处理Ti
正在处理Cu
正在处理O
Processed Dy2TiCuO6 and save

spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 2 tolerance = 9.025000e-03 failed(line 800, /pr

正在处理Nb
正在处理O
Processed SrNbO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/SrNbO3(mp-1173150).csv
3222. 目前化合物是RbHgO3
正在处理Rb
正在处理Hg
正在处理O
Processed RbHgO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/RbHgO3(mp-1186886).csv
3223. 目前化合物是Ba2LaWO6
正在处理Ba
正在处理La
正在处理W
正在处理O
Processed Ba2LaWO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Ba2LaWO6(mp-1522271).csv
3224. 目前化合物是HfFeO3
正在处理Hf
正在处理Fe
正在处理O
Processed HfFeO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/HfFeO3(mp-753856).csv
3225. 目前化合物是AlSbO3
正在处理Al
正在处理Sb
正在处理O
Processed AlSbO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/AlSbO3(mp-1371447).csv
3226. 目前化合物是TmBiO3
正在处理Tm
正在处理Bi
正在处理O
Processed TmBiO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/TmBiO3(mp-755216).csv
3227. 目前化合物是GaFeO3
正在处理Ga
正在处理Fe
正在处理O
Processed GaFeO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/GaFeO3(mp-1184032).csv
3228. 目前化合物是SnPO3
正在处理Sn
正在处理P
正在处理O
Processed SnPO3 and 

spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/spacegroup.c).
spglib: No point group was found (line 405, /private/var/folders/3y/l3fp9ptn18j7k0mgn6pnkk1r0000gn/T/pip-install-xxn6rap4/spglib_8b203eccd15d4d73b8da004394912e84/src/pointgroup.c).
spglib: Attempt 2 tolerance = 9.025000e-03 failed(line 800, /pr

正在处理O
Processed MgCO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/MgCO3(mp-2647164).csv
3832. 目前化合物是TePdO3
正在处理Te
正在处理Pd
正在处理O
Processed TePdO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/TePdO3(mp-979272).csv
3833. 目前化合物是Sr2SmNbO6
正在处理Sr
正在处理Sm
正在处理Nb
正在处理O
Processed Sr2SmNbO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Sr2SmNbO6(mp-1516711).csv
3834. 目前化合物是HoGaO3
正在处理Ho
正在处理Ga
正在处理O
Processed HoGaO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/HoGaO3(mp-755416).csv
3835. 目前化合物是BaTmO3
正在处理Ba
正在处理Tm
正在处理O
Processed BaTmO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/BaTmO3(mp-1183394).csv
3836. 目前化合物是Eu2NbBiO6
正在处理Eu
正在处理Nb
正在处理Bi
正在处理O
Processed Eu2NbBiO6 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/Eu2NbBiO6(mp-1517980).csv
3837. 目前化合物是KNaO3
正在处理K
正在处理Na
正在处理O
Processed KNaO3 and saved to /Users/yangmingyue/Desktop/ABO365/模型应用新新/配位数/KNaO3(mp-973968).csv
3838. 目前化合物是PmMgO3
正在处理Pm
正在处理Mg
正在处理O
Processed Pm

In [10]:
import os
import glob
import pandas as pd


input_folder = "/coordination number/"  
output_folder = "/coordination number change cubic/"  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for csv_file in glob.glob(os.path.join(input_folder, "*.csv")):
    file_name = os.path.basename(csv_file)
    print(f"Processing file: {file_name}")


    df = pd.read_csv(csv_file)


    if 12 in df['cn'].values:
        print(f"File {file_name} contains records with cn=12. Proceeding to check if O's cn is 2.")


        if ((df['ion'] == 'O') & (df['cn'] == 2)).any():
            print(f"File {file_name} contains records where O's cn is 2. Changing O's cn to 6.")


            df.loc[df['ion'] == 'O', 'cn'] = 6
            print(f"O's cn has been changed to 6.")
        else:
            print(f"File {file_name} does not contain records where O's cn is 2. No changes made.")
    else:
        print(f"File {file_name} does not contain records with cn=12. No changes made.")


    output_file_path = os.path.join(output_folder, file_name) # Renamed to avoid conflict with the module 'output_file'
    df.to_csv(output_file_path, index=False)
    print(f"Processed file saved to: {output_file_path}")

print("Processing complete.")

正在处理文件: Ba3ZrIr2O9(mp-1214509).csv
文件 Ba3ZrIr2O9(mp-1214509).csv 中存在 cn=12 的记录，继续检查 O 的 cn 是否为 2。
文件 Ba3ZrIr2O9(mp-1214509).csv 中存在 O 的 cn=2 的记录，将 O 的 cn 修改为 6。
已将 O 的 cn 修改为 6。
文件已保存到 /Users/yangmingyue/Desktop/ABO365/模型应用新新/修改后配位数/Ba3ZrIr2O9(mp-1214509).csv
正在处理文件: Ba2SmWO6(mp-1517191).csv
文件 Ba2SmWO6(mp-1517191).csv 中存在 cn=12 的记录，继续检查 O 的 cn 是否为 2。
文件 Ba2SmWO6(mp-1517191).csv 中不存在 O 的 cn=2 的记录，不做修改。
文件已保存到 /Users/yangmingyue/Desktop/ABO365/模型应用新新/修改后配位数/Ba2SmWO6(mp-1517191).csv
正在处理文件: InRhO3(mp-1203220).csv
文件 InRhO3(mp-1203220).csv 中不存在 cn=12 的记录，不做修改。
文件已保存到 /Users/yangmingyue/Desktop/ABO365/模型应用新新/修改后配位数/InRhO3(mp-1203220).csv
正在处理文件: Mg2SbWO6(mp-1047279).csv
文件 Mg2SbWO6(mp-1047279).csv 中不存在 cn=12 的记录，不做修改。
文件已保存到 /Users/yangmingyue/Desktop/ABO365/模型应用新新/修改后配位数/Mg2SbWO6(mp-1047279).csv
正在处理文件: TbVO3(mp-1187389).csv
文件 TbVO3(mp-1187389).csv 中存在 cn=12 的记录，继续检查 O 的 cn 是否为 2。
文件 TbVO3(mp-1187389).csv 中存在 O 的 cn=2 的记录，将 O 的 cn 修改为 6。
已将 O 的 cn 修改为 6。
文件已保存到 /Users/yangmingyue/Desktop